# Get owned game, sorted by playtime descending

In [2]:
from dotenv import load_dotenv
import os
import requests

In [2]:
playerid = "76561198287996067"

In [3]:
# Load environment variables from .env file
load_dotenv()

# Get the API_KEY from the environment variables
API_KEY = os.getenv('API_KEY')

In [4]:
url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=" + API_KEY + "&steamid=" + playerid + "&format=json"

response = requests.get(url)

if response.status_code != 200:
    print(f"Error: {response.status_code}")
    exit()

data = response.json()
games = data['response']['games']

In [7]:
games[-1]

{'appid': 302190,
 'playtime_forever': 0,
 'playtime_windows_forever': 0,
 'playtime_mac_forever': 0,
 'playtime_linux_forever': 0,
 'playtime_deck_forever': 0,
 'rtime_last_played': 0,
 'playtime_disconnected': 0}

In [ ]:
import urllib.parse

games = sorted(games, key=lambda x: x['playtime_forever'], reverse=True)
games = list(games)[:10]

ACCESS_TOKEN = urllib.parse.quote_plus( os.getenv('ACCESS_TOKEN'))

def get_game_name(appid):
    url = "https://api.steampowered.com/ICommunityService/GetApps/v1/?access_token=" + str(ACCESS_TOKEN)

    for index, app in enumerate(appid):
        url += "&appids%5B" + str(index) + "%5D=" + str(app)

    print(url)
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
    data = response.json()    

    if (len(data['response']) == 0):
        return []
    return [app['name'] for app in data['response']['apps']]

# names = get_game_name([game['appid'] for game in games])
# names

In [ ]:
# if len(names) > 0:
#     print("Top 10 games played by the user:")
#     for index, game in enumerate(games):
#         print(f"{names[index]} - {int(game['playtime_forever']/60)} hours")

Top 10 games played by the user:
Terraria - 1192 hours
Don't Starve Together - 197 hours
Robocraft - 192 hours
Blender - 138 hours
Left 4 Dead 2 - 134 hours
Counter-Strike 2 - 118 hours
DOOM Eternal - 112 hours
Noita - 96 hours
tModLoader - 88 hours
DOOM - 77 hours


# Prepare demo data

In [9]:
demo_players = [76561198027214426, 76561197967404516, 76561198049101130, 76561199081881137]
# demo_players = [76561198389473114]

import pandas as pd

purchased_games = pd.read_csv('./clean_datasets/purchased_games.csv')
purchased_games['library'] = purchased_games['library'].apply(eval)

# get the top 10 games played by the demo players
for player in demo_players:
    url = "https://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key=" + API_KEY + "&steamid=" + str(player) + "&format=json"

    response = requests.get(url)

    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        exit()

    data = response.json()
    games = data['response']['games']

    games = sorted(games, key=lambda x: x['playtime_forever'], reverse=True)
    games = list(games)[:10]

    if len(games) == 0:
        print("No games found for this user.")
    else:
        print("Top 10 games played by the user:")
        for index, game in enumerate(games):
            print(f"{index + 1}. {game['appid']} - {int(game['playtime_forever']/60)} hours")
        
        top_10 = [game['appid'] for game in games]
        
        owned_games = purchased_games.loc[purchased_games['playerid'] == player, 'library'].values[0]
        print(len(owned_games))
        owned_games_for_demo = [game for game in owned_games if game not in top_10]
        print(len(owned_games_for_demo))
        purchased_games.loc[purchased_games['playerid'] == player, 'library'] = str(owned_games_for_demo)
        
purchased_games.to_csv('./clean_datasets/purchased_games_demo.csv', index=False)
print("Updated purchased games dataset with demo players' data.")

Top 10 games played by the user:
1. 4000 - 8722 hours
2. 72200 - 8674 hours
3. 252490 - 3971 hours
4. 1250 - 1077 hours
5. 251570 - 960 hours
6. 232090 - 773 hours
7. 218620 - 699 hours
8. 42690 - 568 hours
9. 675560 - 500 hours
10. 475150 - 500 hours
25100
25090
Top 10 games played by the user:
1. 740250 - 3871 hours
2. 1173510 - 2522 hours
3. 908520 - 1370 hours
4. 365670 - 1199 hours
5. 1068820 - 558 hours
6. 99900 - 395 hours
7. 362890 - 286 hours
8. 620980 - 212 hours
9. 113400 - 155 hours
10. 438100 - 151 hours
1578
1568
Top 10 games played by the user:
1. 221100 - 1231 hours
2. 107410 - 1109 hours
3. 730 - 688 hours
4. 227300 - 492 hours
5. 271590 - 265 hours
6. 244210 - 225 hours
7. 202990 - 186 hours
8. 226700 - 184 hours
9. 1938090 - 86 hours
10. 431960 - 57 hours
489
479
Top 10 games played by the user:
1. 236390 - 819 hours
2. 730 - 588 hours
3. 700330 - 138 hours
4. 271590 - 132 hours
5. 4000 - 94 hours
6. 304930 - 93 hours
7. 1407200 - 83 hours
8. 707010 - 68 hours
9. 443

In [29]:
purchased_games.loc[purchased_games['playerid'] == 76561199081881137, 'library']

53666    [230410, 223710, 262410, 265630, 218620, 22110...
Name: library, dtype: object

In [8]:
# check if the game is in the library of this user
playerid = 76561198389473114
gameid = 730
purchased_games.loc[purchased_games['playerid'] == playerid, 'library'].values[0]


'[251570, 2923300, 377160, 700580, 225540, 427730, 107410, 431960, 391220, 265930, 433340, 433910, 438100, 274170, 441870, 274190, 444090, 596350, 365590, 524440, 280180, 282660, 450670, 284160, 457330, 285900, 290930, 222900]'